# SDG 6.6.1: Analysis and Reporting

## Extracting GSWE data

In [1]:
import ee
from GSWE_reporting import extract_gswe
ee.Authenticate()

Enter verification code: 4/2wGV3zNgpMoIxr4dLR-nmMH39cMWuE1P605GiTzWxj2jWADrVZXtqXg

Successfully saved authorization token.


In [2]:
ee.Initialize()  

In [3]:
geometry = ee.Geometry.Polygon(
        [[[-12.598544729822379, 61.78863494939058],
          [-12.598544729822379, 49.00174346741333],
          [3.749111520177621, 49.00174346741333],
          [3.749111520177621, 61.78863494939058]]])

In [4]:
extract_gswe('JRC/GSW1_2/YearlyHistory', geometry, 1984, 2020, 'GSWE_exports')

GSWE files are being extracted to GSWE_exports in your Google Drive


---
## Preparing HydroBASIN boundary

In [5]:
import geopandas as gpd
from shapely.ops import unary_union
from GSWE_reporting import clip_basin_to_boundary

In [6]:
eu_hydrobasin = gpd.read_file('./boundaries/hybas_eu_lev06_v1c.shp')
uk_boundary = gpd.read_file('./boundaries/Countries_December_2018_Boundaries_UK_BFC.shp') 

In [7]:
target_crs = ('+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')
osgb_proj4 = ('+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +datum=OSGB36 +units=m +no_defs')
uk_boundary.crs = osgb_proj4  

In [8]:
hydro_clipped = clip_basin_to_boundary(eu_hydrobasin, uk_boundary, target_crs)

union national boundary


In [9]:
hydro_clipped = hydro_clipped.drop(hydro_clipped.columns[1:-1], axis=1) 
hydro_clipped.to_file('./boundaries/UK_hydrobasin_UTM.shp')

---
## Preparing GSWE outputs

In [1]:
import geopandas as gpd
from GSWE_reporting import get_gswe_paths, reproject_GSWE

In [2]:
gsw_files = get_gswe_paths('./GSW_output/', '.tif')

In [4]:
reproject_GSWE(gsw_files, '+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')

---
## Calculating Water Extent

In [2]:
from GSWE_reporting import surface_water_extent

In [3]:
gsw_file_path_list = get_gswe_paths('./Reprojected/', 'UTM.tif')

In [4]:
hydro_basin = gpd.read_file('./boundaries/UK_hydrobasin_UTM.shp')

In [5]:
water_extent = surface_water_extent(gsw_file_path_list, hydro_basin)

Calculating water extent in 1984
Calculating water extent in 1985
Calculating water extent in 1986
Calculating water extent in 1987
Calculating water extent in 1988
Calculating water extent in 1989
Calculating water extent in 1990
Calculating water extent in 1991
Calculating water extent in 1992
Calculating water extent in 1993
Calculating water extent in 1994
Calculating water extent in 1995
Calculating water extent in 1996
Calculating water extent in 1997
Calculating water extent in 1998
Calculating water extent in 1999
Calculating water extent in 2000
Calculating water extent in 2001
Calculating water extent in 2002
Calculating water extent in 2003
Calculating water extent in 2004
Calculating water extent in 2005
Calculating water extent in 2006
Calculating water extent in 2007
Calculating water extent in 2008
Calculating water extent in 2009
Calculating water extent in 2010
Calculating water extent in 2011
Calculating water extent in 2012
Calculating water extent in 2013
Calculatin

In [14]:
water_extent.to_csv('./water_type_by_basin.csv', index = False)

### National Water Extent

In [6]:
import pandas as pd
import altair as alt

In [7]:
water_extent['HYBAS_ID'] = water_extent["HYBAS_ID"].apply(str)

In [8]:
baseline = water_extent[water_extent['Year'] == 'baseline']
water_type_df = water_extent[water_extent['Year'] != 'baseline'] 
water_type_by_year = water_type_df.groupby(['Year'], as_index=False).sum() 

In [12]:
water_type_melt = water_type_by_year.melt('Year', value_name = 'vals', var_name='variable')

alt.Chart(water_type_melt).mark_bar(opacity=0.7).encode(
    x=alt.X('Year:N', title = 'Year'),
    y=alt.Y('sum(vals)', title = 'Water Extent (km\N{SUPERSCRIPT TWO})'),
    color=alt.Color('variable',
        scale = alt.Scale(domain=['Permanent', 'Ephemeral', 'Seasonal'],
                          range=['#1f78b4', '#b2df8a', '#a6cee3']),
        title = 'Water Type'),
    order=alt.Order('sum(variable)',sort='ascending'),
    tooltip = ['Year:N', 
               alt.Tooltip('vals:Q', title = 'Extent')]
).interactive()

alt.Chart(...)

In [13]:
water_type_by_year['HYBAS_ID'] = 'Total'
water_type_by_basin = water_type_df.append(water_type_by_year, sort=False, ignore_index = True)
water_type_by_basin.to_csv('./water_type_w_total.csv', header = True, index = False) 